# Assembly: instruction formats; mapping assembly code to C code
_COSC 208, Introduction to Computer Systems, 2023-03-20_

## Announcements
* Project 3 due Thursday @ 11pm

## Outline
* Mid-semester course evaluation
* Warm-up
* Assembly
* Operands
* Mapping assembly code to C code

## Mid-semester Course Evaluation (separate sheet)
Your feedback on this course is important for helping Aaron improve the learning environment. Thank you for your thoughtful, anonymous responses.

## Warm-up

Q1: _Where are multiplexers used in a processing unit?_

* To select which registers' values are input to the arithmetic logic unit (ALU)
* To select which operation's result is output by the ALU
* A demultiplexer is used to select which register the result is stored in

<p style="height:8em;"></p>

Q2: _How do multiplexers relate to assembly instructions?_

* Some bits determine the operation – used as select bits for the multiplexer between the arithmetic and logic circuits and the ALU's output
* Some bits determine the first source register and other bits determine the second source register – used as select bits for the multiplexers between the register file's outputs and ALU's inputs
* Some bits determine the destination register – used as select bits for the demultiplexer between the ALU's output and register file's input

<p style="height:8em;"></p>

🛑 **STOP here** after completing the above questions; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Assembly

* Instruction set architectures (ISAs) — defines the low-level instructions a central processing unit (CPU), or processor, can execute
    * Most popular Complex Instruction Set Computer (CISC) architecture: x86 (includes IA32 and x86-64)
    * Most popular Reduced Instruction Set Computer (RISC) architecture: ARM
    * Other common RISC architectures: SPARC (Scalable Processor Architecture), MIPS (Microprocessor without Interlocked Pipelined Stages), PowerPC, ARC (Argonaut RISC Core)

* Example program (`seasons.c`)

In [1]:
/* 1*/   #include <stdio.h>
/* 2*/   int seasons_of_love() {
/* 3*/       int dpy = 365;
/* 4*/       int hpd = 24;
/* 5*/       int mph = 60;
/* 6*/       int h = dpy * hpd;
/* 7*/       int m = h * mph;
/* 8*/       return m;
/* 9*/   }
/*10*/  int main() {
/*11*/      int minutes = seasons_of_love();
/*12*/      printf("%d minutes\n", minutes);
/*13*/      printf("%d moments so dear\n", minutes - 600);
/*14*/      printf("%d minutes\n", minutes);
/*15*/      printf("How do you measure? Measure a year?\n");
/*16*/  }

525600 minutes
525000 moments so dear
525600 minutes
How do you measure? Measure a year?


* Viewing assembly code
    * Compile: `gcc -fomit-frame-pointer -o seasons seasons.c`
    * Disassemble executable: `objdump -d seasons > seasons.txt`

* Assembly code (excerpt from `seasons.txt`)
    ```
    00000000000007ac <seasons_of_love>:
        7ac:   d10083ff        sub     sp, sp, #0x20
        7b0:   52802da0        mov     w0, #0x16d  
        7b4:   b9000fe0        str     w0, [sp, #12]
        7b8:   52800300        mov     w0, #0x18   
        7bc:   b90013e0        str     w0, [sp, #16]
        7c0:   52800780        mov     w0, #0x3c   
        7c4:   b90017e0        str     w0, [sp, #20]
        7c8:   b9400fe1        ldr     w1, [sp, #12]
        7cc:   b94013e0        ldr     w0, [sp, #16]
        7d0:   1b007c20        mul     w0, w1, w0
        7d4:   b9001be0        str     w0, [sp, #24]
        7d8:   b9401be1        ldr     w1, [sp, #24]
        7dc:   b94017e0        ldr     w0, [sp, #20]
        7e0:   1b007c20        mul     w0, w1, w0
        7e4:   b9001fe0        str     w0, [sp, #28]
        7e8:   b9401fe0        ldr     w0, [sp, #28]
        7ec:   910083ff        add     sp, sp, #0x20
        7f0:   d65f03c0        ret
    ```

<div style="page-break-after:always;"></div>

* _What do each of the columns contain?_
    * Code memory address
    * Bytes corresponding to instruction
    * Operation
    * Operands

## Operands
* Registers
    * General purpose registers: 31
        * `w0` through `w30` (32-bit) 
        * `x0` through `x30` (64-bit)
    * Stack pointer (top of current stack frame): `sp`
        * Always holds a memory address
* Constant
    * In hex — e.g., `#0x20`
    * In decimal — e.g., `#8`
* Memory
    * Dereference — e.g., `[x1]`
    * Add to memory address, then dereference — e.g., `[sp,#16]`

## Mapping assembly code to C code

* Mathematical operation pattern
    * load (ldr) first operand from the stack into a register OR move (mov) first operand into a regster
    * load (ldr) second operand from the stack into a register OR move (mov) second operand into a regster
    * compute
    * store (str) resulting value from a register onto the stack
* Local variable initialization pattern
    * move (mov) constant value into register
    * store (str) register value onto the stack
* Function begin pattern
    * create stack frame by decreasing (sub) stack pointer
    * store (str) parameters from registers x/w0, x/w1, etc. onto the stack
* Function return/end pattern
    * load (ldr) return value from the stack into the w/x0 register
    * destroy stack frame by increasing (add) stack pointer
    * return (ret) to caller

* Example

```
00000000000007ac <seasons_of_love>:
    7ac:   d10083ff        sub     sp, sp, #0x20    // Line 2   Create stack frame
    7b0:   52802da0        mov     w0, #0x16d       // Line 3   Local variable (dpy) initialization 
    7b4:   b9000fe0        str     w0, [sp, #12]    //   V
    7b8:   52800300        mov     w0, #0x18        // Line 4   Local variable (hpd) initialization
    7bc:   b90013e0        str     w0, [sp, #16]    //   V
    7c0:   52800780        mov     w0, #0x3c        // Line 5   Local variable (mph) initialization
    7c4:   b90017e0        str     w0, [sp, #20]    //   V
    7c8:   b9400fe1        ldr     w1, [sp, #12]    // Line 6   Mathematical operation
    7cc:   b94013e0        ldr     w0, [sp, #16]    //   |
    7d0:   1b007c20        mul     w0, w1, w0       //   |
    7d4:   b9001be0        str     w0, [sp, #24]    //   V
    7d8:   b9401be1        ldr     w1, [sp, #24]    // Line 7   Mathematical operation
    7dc:   b94017e0        ldr     w0, [sp, #20]    //   |
    7e0:   1b007c20        mul     w0, w1, w0       //   |
    7e4:   b9001fe0        str     w0, [sp, #28]    //   V
    7e8:   b9401fe0        ldr     w0, [sp, #28]    // Line 8   Function return/end
    7ec:   910083ff        add     sp, sp, #0x20    //   |
    7f0:   d65f03c0        ret                      //   V
```

## Extra practice

* Q3: _Draw a processor with 8 general purpose registers and two operations (add and subtract)._

![](images/circuits/processing_unit_empty.png)